In [1]:
import pandas as pd
import pypsa
import numpy as np
network = pypsa.Network()

In [2]:
bus = pd.read_csv('data/base_grid/bus.csv')
sub = pd.read_csv('data/base_grid/sub.csv')
sub = sub.drop(columns = 'interconnect')
bus2sub = pd.read_csv('data/base_grid/bus2sub.csv')
bus2sub = bus2sub.drop(columns = 'interconnect')

In [3]:
buses = bus.merge(bus2sub, how = 'left', on='bus_id')
buses = buses.merge(sub,how='left',on='sub_id')

In [4]:
buses['Va'] = buses['Va'].apply(np.deg2rad)
buses['control'] = buses['type'].map({1:'PQ',2:'PV'})
buses.drop(inplace=True,columns = ['type'])
buses.dropna(subset=['control'],inplace=True)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [5]:
buses = buses.set_index('bus_id')
buses_columns = {'lat':'y',
                                'lon':'x',
                                'Vm':'v_mag_pu',
                                'Pd':'p',
                                'Qd':'q',
                                'Va':'v_ang',
                                'baseKV':'v_nom',
                                'Vmax':'v_mag_pu_max',
                                'Vmin':'v_mag_pu_min'}

buses = buses.rename(columns = buses_columns)
buses = buses[buses_columns.values()]

In [6]:
buses

,y,x,v_mag_pu,p,q,v_ang,v_nom,v_mag_pu_max,v_mag_pu_min
bus_id,,,,,,,,,
1,43.9761,-70.2211,1.03511,0.00,0.00,-1.124168,230.0,1.1,0.9
2,43.9761,-70.2211,1.03706,0.00,0.00,-1.139032,115.0,1.1,0.9
3,43.9761,-70.2211,1.03293,9.97,1.30,-1.165906,18.0,1.1,0.9
4,43.9761,-70.2211,1.03744,0.00,0.00,-1.140197,1.0,1.1,0.9
5,43.9761,-70.2211,1.03607,0.00,0.00,-1.124050,230.0,1.1,0.9
...,...,...,...,...,...,...,...,...,...
3008156,31.0919,-96.6950,1.02611,0.00,0.00,-0.804766,230.0,1.1,0.9
3008157,31.0919,-96.6950,1.02808,0.00,0.00,-0.872862,115.0,1.1,0.9
3008158,30.7217,-96.4608,1.02650,0.00,0.00,-0.688833,500.0,1.1,0.9


In [7]:
network.import_components_from_dataframe(buses,"Bus")

In [8]:
generators = pd.read_csv('data/base_grid/plant.csv')
cost = pd.read_csv('data/base_grid/gencost.csv')
hydro = pd.read_csv('data/base_grid/hydro.csv')
solar = pd.read_csv('data/base_grid/solar.csv')
wind = pd.read_csv('data/base_grid/wind.csv')
outputs = solar.merge(hydro).merge(wind)
generators = generators.merge(cost.drop(columns = ['type','interconnect']),how='left',on='plant_id')
generators = generators.merge(bus.drop(columns = ['type', 'interconnect']),how='left',on='bus_id')
generators = generators.merge(bus[['bus_id','type']].rename(columns = {'type':'control'}),how='left',on='bus_id')


In [9]:
def output(row):
    try:
        return outputs[str(row['plant_id'])]
    except Exception as e:
        return np.nan

In [10]:
generators['output'] = generators.apply(lambda x: output(x), axis=1)

In [11]:
pop = generators[generators['output'].isna()].drop(columns = ['output']).nunique(dropna=False) == generators[generators['output'].notna()].drop(columns = ['output']).nunique(dropna=False)

In [12]:
power_as_series = generators['Pg'].apply(lambda x: [x]*8784)
generators['output'].fillna(power_as_series,inplace=True)

In [13]:
generators.set_index('plant_id',inplace=True)
generators['control'] = generators['control'].map({1:'PQ',2:'PV'})
generator_rename = {'bus_id':'bus',
                    'output':'p',
                    'Pg':'p_nom',
                    'Pmin':'p_nom_min',
                    'Pmax':'p_nom_max',
                    'type':'carrier',
                    'startup':'start_up_cost',
                    'shutdown':'shut_down_cost',
                    'marginal_cost':'c1',
                    'Qg':'q'}
generators.rename(columns = generator_rename,inplace=True)


In [14]:
generators

,bus,p_nom,q,Qmax,Qmin,Vg,mBase,status,p_nom_max,p_nom_min,...,baseKV,loss_zone,Vmax,Vmin,lam_P,lam_Q,mu_Vmax,mu_Vmin,control,p
plant_id,,,,,,,,,,,,,,,,,,,,,
0,845,186.58,40.25,95.08,-20.73,1.0090,224.16,1,177.194,34.167803,...,13.8,1,1.1,0.9,63.95,0,0,0,PV,"[186.58, 186.58, 186.58, 186.58, 186.58, 186.5..."
1,850,211.25,32.45,98.56,-25.80,1.0488,253.80,1,206.883,81.227085,...,18.0,1,1.1,0.9,63.95,0,0,0,PV,"[211.25, 211.25, 211.25, 211.25, 211.25, 211.2..."
2,851,211.25,53.96,98.56,-25.80,1.0500,253.80,1,206.883,78.938172,...,13.8,1,1.1,0.9,63.95,0,0,0,PV,"[211.25, 211.25, 211.25, 211.25, 211.25, 211.2..."
3,852,211.25,53.57,98.56,-25.80,1.0545,253.80,1,206.883,39.752755,...,13.8,1,1.1,0.9,63.95,0,0,0,PV,"[211.25, 211.25, 211.25, 211.25, 211.25, 211.2..."
4,853,211.25,32.45,98.56,-25.80,1.0488,253.80,1,206.883,102.629405,...,13.8,1,1.1,0.9,63.95,0,0,0,PV,"[211.25, 211.25, 211.25, 211.25, 211.25, 211.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14045,3001028,0.00,0.00,0.00,0.00,1.0000,200.00,1,155.400,0.000000,...,115.0,9,1.1,0.9,0.00,0,0,0,PQ,0 73.460236 1 86.621887 2 85...
14046,3002048,0.00,0.00,0.00,0.00,1.0000,200.00,1,151.200,0.000000,...,161.0,10,1.1,0.9,0.00,0,0,0,PQ,0 40.790739 1 70.110531 2 74...
14047,3003142,0.00,0.00,0.00,0.00,1.0000,200.00,1,150.000,0.000000,...,115.0,27,1.1,0.9,0.00,0,0,0,PQ,0 41.609568 1 53.958899 2 66...


In [17]:
network.import_components_from_dataframe(generators,"Generator")

Index(['4820', '12168', '13253'], dtype='object', name='plant_id')
ERROR:pypsa.io:Error, new components for Generator are not unique


In [18]:
network.generators

,bus,p_nom,Qmax,Qmin,Vg,mBase,p_nom_max,p_nom_min,Pc1,Pc2,...,14040,14041,14042,14043,14044,14045,14046,14047,14048,14049
0,845,186.58,95.08,-20.73,1.0090,224.16,177.194,34.167803,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,850,211.25,98.56,-25.80,1.0488,253.80,206.883,81.227085,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,851,211.25,98.56,-25.80,1.0500,253.80,206.883,78.938172,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,852,211.25,98.56,-25.80,1.0545,253.80,206.883,39.752755,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,853,211.25,98.56,-25.80,1.0488,253.80,206.883,102.629405,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ramp_limit_up,,0.00,NaN,NaN,NaN,NaN,inf,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ramp_limit_down,,0.00,NaN,NaN,NaN,NaN,inf,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ramp_limit_start_up,,0.00,NaN,NaN,NaN,NaN,inf,0.000000,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ramp_limit_shut_down,,0.00,NaN,NaN,NaN,NaN,inf,0.000000,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [19]:
branches = pd.read_csv('data/base_grid/branch.csv').drop(columns = ['interconnect'])

In [ ]:
branches.rename(columns={
    "branch_id": "name",
    "from_bus_id": "bus0",
    "to_bus_id": "bus1",
    "angmin": "v_ang_min",
    "angmax": "v_ang_max"
},inplace=True
               )

In [ ]:
network.import_components_from_dataframe(branches,"Line")

In [ ]:
pop = network.graph()

In [1]:
import networkx as nx
nx.draw(pop)

NameError: name 'pop' is not defined